In [2]:
import pandas as pd
import numpy as np




In [3]:
df= pd.read_csv("dataset.csv")

In [4]:
df.head()

,text,emoji
0,all far. scared omg Look,😱
1,ball. tears Support,😢
2,hurt miss Concern.,😢
3,machine. hurt cry religious Energy,😢
4,beautiful while. Matter,😍


ADDING WORD2VEC


In [5]:
from gensim.utils import simple_preprocess

In [6]:
tokenized = df["text"].apply(simple_preprocess)


In [7]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(sentences=tokenized,vector_size=100,window=5, min_count=1, workers=4)

In [8]:
word_index = {word: i+1 for i, word in enumerate(w2v_model.wv.index_to_key)}

Word index -> sequence


In [9]:
sequences = [[word_index.get(word,0) for word in sentence] for sentence in tokenized]

In [10]:
#so like summarizing -> we had df of sentences -> converted into tokens of sentences
#-> then gave each word a index -> then converted the embedded layer ,to look into word in embedded matrix
#then converted the word indexes into sequence -> back to df

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequences= pad_sequences(sequences,padding='post')

In [22]:
padded_sequences.shape

(24000, 10)

SETTING OUTPUT in numbers


In [13]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [14]:
df.columns

Index(['text', 'emoji'], dtype='object')

In [16]:
df["emoji"]=le.fit_transform(df["emoji"])

mapping = dict(zip(le.classes_, range(len(le.classes_))))

# Display the mapping
print(mapping)

{'😇': 0, '😊': 1, '😍': 2, '😎': 3, '😡': 4, '😢': 5, '😱': 6, '😴': 7, '🤒': 8, '🤓': 9, '🤔': 10, '🤗': 11, '🤯': 12, '🥳': 13}


In [17]:
# {'😇': 0, '😊': 1, '😍': 2, '😎': 3, '😡': 4, '😢': 5, '😱': 6, '😴': 7,
#  '🤒': 8, '🤓': 9, '🤔': 10, '🤗': 11, '🤯': 12, '🥳': 13}

SETTING UP MODEL

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

model =Sequential()
model.add(Embedding(
    input_dim=24000,
     output_dim=10,

    input_length=10,
    trainable=True
))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(14, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [30]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [31]:

# Train the model
model.fit(padded_sequences, df["emoji"], epochs=10)

Epoch 1/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5989 - loss: 1.3401
Epoch 2/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9929 - loss: 0.0342
Epoch 3/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9973 - loss: 0.0122
Epoch 4/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9984 - loss: 0.0068
Epoch 5/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9993 - loss: 0.0027
Epoch 6/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9993 - loss: 0.0022
Epoch 7/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9998 - loss: 0.0022
Epoch 8/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9990 - loss: 0.0029
Epoch 9/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9994 - loss: 0.0019
Epoch 10/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9997 - loss: 8.5146e-04


In [32]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (32, 10, 10)           │       240,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (32, 100)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (32, 128)              │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (32, 64)               │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (32, 14)               │           910 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 786,284 (3.00 MB)

 Trainable params: 262,094 (1023.80 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 524,190 (2.00 MB)

In [41]:
text="sad"
token=simple_preprocess(text)
sequence=[word_index.get(word,0) for word in token]
padded=pad_sequences([sequence],padding='post',maxlen=10)
prediction = model.predict(padded)

predicted_index = prediction.argmax(axis=1)[0]
emoji_dict = {0: '😇', 1: '😊', 2: '😍', 3: '😎', 4: '😡', 5: '😢',
              6: '😱', 7: '😴', 8: '🤒', 9: '🤓', 10: '🤔', 11: '🤗',
              12: '🤯', 13: '🥳'}

predicted_emoji = emoji_dict[predicted_index]
print("Predicted Emoji:", predicted_emoji)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Emoji: 😢
